# Production des occupations renseignées dans Wikidata 

Dans ce carnet est proposée la méthode dep roduction des occupations de la population Wikidata retenue.


In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
### https://mwouts.github.io/itables/quick_start.html

from itables import init_notebook_mode, show

init_notebook_mode(all_interactive=False)


In [4]:
### Librairies déjà installées avec Python
import pprint
import csv
import sys

import sqlite3 as sql

import time
import datetime
from dateutil import parser

from importlib import reload
from shutil import copyfile


In [5]:
### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
sys.path.insert(0, '../..')


import sparql_functions as spqf

## Préparer les données

### SPARQL Query qui récupère les données

On exécute la requête grâce à une fonction de la librairie locale qui réalise la mise en forme

In [6]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [7]:
query = """
SELECT DISTINCT ?item ?occupation ?occupationLabel
        WHERE {
            {
            {?item wdt:P106 wd:Q169470}
            # UNION
            # {?item wdt:P101 wd:Q413}  
          UNION 
          {?item wdt:P106 wd:Q11063}
            # UNION
            # {?item wdt:P101 wd:Q333} 
            UNION
              {?item wdt:P106 wd:Q155647}
            # UNION
            # {?item wdt:P101 wd:Q34362} 
              }
          
          ?item wdt:P31 wd:Q5;  # Any instance of a human.
              wdt:P569 ?birthDate;
                wdt:P106 ?occupation.
        BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?year)
        FILTER(xsd:integer(?year) > 1350 )
          
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        } 
     ORDER BY ?year
"""

In [8]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

In [9]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

67587


[['http://www.wikidata.org/entity/Q855257',
  'http://www.wikidata.org/entity/Q155647',
  'astrologer'],
 ['http://www.wikidata.org/entity/Q855257',
  'http://www.wikidata.org/entity/Q170790',
  'mathematician'],
 ['http://www.wikidata.org/entity/Q855257',
  'http://www.wikidata.org/entity/Q1622272',
  'university teacher']]

In [10]:
### Créer un DataFrame à partir du résultat
df_r = pd.DataFrame(r)
df_r.columns = ['personUri', 'occupationUri', 'occupationLabel']
df_r.head()

,personUri,occupationUri,occupationLabel
0,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q155647,astrologer
1,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q170790,mathematician
2,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q1622272,university teacher
3,http://www.wikidata.org/entity/Q855257,http://www.wikidata.org/entity/Q4964182,philosopher
4,http://www.wikidata.org/entity/Q4121967,http://www.wikidata.org/entity/Q11063,astronomer


In [11]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67587 entries, 0 to 67586
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   personUri        67587 non-null  object
 1   occupationUri    67587 non-null  object
 2   occupationLabel  67587 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


### Créer une nouvelle table dans la base de données SQLite

On va stocker dans cette table le résultat de la requête SPARQL préalablement transformé en DataFrame

In [12]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/astronomers_import.db')
cn

In [ ]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_r.to_sql(name='wdt_person_occupation', con=cn, if_exists='fail')
except Exception as e:
    print('Erreur: ',  e)

In [15]:
### Vérifier que les données ont été importées correctement
cur = cn.cursor()
l = cur.execute("SELECT * FROM wdt_person_occupation limit 3").fetchall()
### On a mis le résultat de la requête SQL 
# dans une liste 'l' qu'on affiche avec une boucle 'for'
# dans le cadre d'une 'list comprehension'
a = [print(e) for e in l]

(0, 'http://www.wikidata.org/entity/Q855257', 'http://www.wikidata.org/entity/Q155647', 'astrologer')
(1, 'http://www.wikidata.org/entity/Q855257', 'http://www.wikidata.org/entity/Q170790', 'mathematician')
(2, 'http://www.wikidata.org/entity/Q855257', 'http://www.wikidata.org/entity/Q1622272', 'university teacher')


In [30]:
### Vérifier que les données ont été importées correctement
cur = cn.cursor()
l = cur.execute("SELECT COUNT(*) FROM wdt_person_occupation ").fetchone()
print('Nombre de lignes de la table:', l[0])

Nombre de lignes de la table: 67587


### Créer une table qui contient les métiers 

In [16]:
q="""
-- noter que la fonction TRIM élimine les éventuels espaces en début ou fin de chaine de charactères
-- la fonction LOWER met tout au minuscule
SELECT TRIM(occupationUri) as occupationUri, LOWER(TRIM(occupationLabel)) AS occupationLabel, COUNT(*) as effectif
FROM wdt_person_occupation
GROUP BY TRIM(occupationUri), LOWER(TRIM(occupationLabel))
ORDER BY effectif DESC
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
rdf = pd.DataFrame(r,columns=['uri', 'label', 'freq'])
rdf.head()

,uri,label,freq
0,http://www.wikidata.org/entity/Q169470,physicist,25564
1,http://www.wikidata.org/entity/Q11063,astronomer,8054
2,http://www.wikidata.org/entity/Q1622272,university teacher,7423
3,http://www.wikidata.org/entity/Q170790,mathematician,2888
4,http://www.wikidata.org/entity/Q36180,writer,1122


In [17]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uri     1246 non-null   object
 1   label   1246 non-null   object
 2   freq    1246 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 29.3+ KB


In [20]:
### Après avoir crée avec SQLite Studio une table 'wdt_occupation'
# la remplir avec une ligne par métier


q2 = """
INSERT INTO wdt_occupation (wdt_uri,label)
SELECT DISTINCT TRIM(occupationUri), LOWER(TRIM(occupationLabel))
FROM wdt_person_occupation;
"""

## Attention : la requête est commentée pour éviter 
# de la réexécuter par mégarde: en effet, cette table
# servira au codage des domaines — ne pas l'effacer

cur = cn.cursor()
# r = cur.execute(q2)
# cn.commit()

In [19]:
### ATTENTION : ces requêtes vident la table wdt_occupation et remettent 
# la séquence des primary keys au début: la première valeur sera donc 1

### ATTENTION 2: cette table permet le codage des domaines — ne pas l'effacer !

q3 = """
DELETE FROM wdt_occupation  ;
"""

q4="""UPDATE SQLITE_SEQUENCE 
    SET seq = 0
    WHERE name ='wdt_occupation';
    """

### ATTENTION : cette cellule contient le code nécessaire si on doit refaire la table
# généralement on ne doit pas l'exécuter, et si on veut vraiment l'exécuter  
# il faut décommenter le code

# cur = cn.cursor()
# r = cur.execute(q3)
# r = cur.execute(q4)
# cn.commit()

## Vérifier l'importation

In [21]:
### Le nombre de métiers différents: noter qu'il y des répétitions
## et inconsistances dans les données de Wikidata

q="""
SELECT COUNT(*)
FROM wdt_occupation
"""
cur = cn.cursor()
r = cur.execute(q).fetchone()
print('Nombre de métiers différents:' , r[0])

Nombre de métiers différents: 1246


In [22]:
## Vérifier que le DataFrame rdf contient le même nombre de modalités
print(len(rdf))

1246


In [26]:
### Quelques métiers, 
# pour mieux explorer parcourir la table dans DBeaver

q="""
SELECT *
FROM wdt_occupation
LIMIT 5
"""
cur = cn.cursor()
r = cur.execute(q).fetchall()
pprint.pprint(r)

[(1, 'astrologer', None, 'http://www.wikidata.org/entity/Q155647', None),
 (2, 'mathematician', None, 'http://www.wikidata.org/entity/Q170790', None),
 (3,
  'university teacher',
  None,
  'http://www.wikidata.org/entity/Q1622272',
  None),
 (4, 'philosopher', None, 'http://www.wikidata.org/entity/Q4964182', None),
 (5, 'astronomer', None, 'http://www.wikidata.org/entity/Q11063', None)]
